In [ ]:

#!pip install flask-ngrok

In [ ]:
import nltk
nltk.download('punkt')

import json
import numpy as np
from keras.models import load_model  #USE JOBLIB instead
import random
import pickle

from googlesearch import search
import requests
from lxml import html
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:


#load pickled model
model = load_model("drive/My Drive/phoebe/chatbot_model.h5")
#load files
intents = json.loads(open('drive/My Drive/phoebe/intents.json').read())
words = pickle.load(open('drive/My Drive/phoebe/words.pkl','rb'))
classes = pickle.load(open('drive/My Drive/phoebe/classes.pkl','rb'))

#define our global variable
global context
context = {'userID': 'none'}


In [ ]:
#function
def bow(sentence, words, show_details=True): #show_details = false for deployment
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    #print(return_list)
    return return_list

#google weather function
def search_weather(location):
  answer = "The weather in "+location+" is "
  query = "Weather in "+location+" today"
  try:
    article = ''
    search_result_list = list(search(query, lang='en', num=10, pause=1))
    page = requests.get(search_result_list[0]) #scraping weather.com
    
    if page.status_code == 200:
      soup = BeautifulSoup(page.text, features='lxml')
      
      #all_articles = soup.find('div', {"class" : "_-_-components-src-organism-CurrentConditions-CurrentConditions--primary--2DOqs" })
      all_articles = soup.find('span', {"class": "CurrentConditions--tempValue--3KcTQ"})
      if all_articles != None:
        result = all_articles.text  #only temperature
        ar = soup.find('div', {'data-testid': "precipPhrase"}).text  #details
        
        return answer+result+' with '+str(ar)
      else: 
        raise Exception("Sorry, facing an error")
  
  except Exception as e:
    print(str(e))
    result = "I can not find an answer for that. Try a different query!"
    return result


#function that calls other google search functions
google_search_func = {'Search weather': search_weather}#, 'search celebrity': search_celebrity} #argument is passed when called

#driver function version #2
def chatbot_response_2(msg, userID='userID', show_details = False):
    results = predict_class(msg, model)

    if context['userID'] is 'none':
      if results:
          # loop as long as there are matches to process
          while results:
            for i in intents['intents']:
                  #print("chatbot_response i : ", i)
                  # find a tag matching the first result
                  if i['tag'] == results[0]['intent']:
                      # set context for this intent if necessary
                      if 'context' in i:
                          if show_details: print ('context:', i['context'])
                          context['userID'] = i['context']

                      # check if this intent is contextual and applies to this user's conversation
                      
                      if not 'context_filter' in i or (userID in context and 'context_filter' in i and i['context_filter'] == context['userID']):
                          if show_details: print ('tag:', i['tag'])
                          # a random response from the intent
                          return print(random.choice(i['responses']))

            results.pop(0)
    else:
      for i in intents['intents']:
        if context['userID'] in i['tag']:
           print(i['responses'][0])
           msg = input()
           return google_search_func[i['tag']](msg)  #call based on tag
           #have to respond and then call google search function acc to context


HTTP Error 429: Too Many Requests


In [ ]:
#TEST--------------------------------------------------------------------------------------------
user_query = "What is the weather?"
print(chatbot_response_2(user_query))

Checking...Bangalore
'NoneType' object has no attribute 'text'
I can not find an answer for that. Try a different query!
